In [134]:
from selenium import webdriver
import pandas as pd
from bs4 import BeautifulSoup
import time
import numpy as np
from tqdm import tqdm

In [198]:
df = pd.read_csv('data.csv')

In [199]:
df.head()

,title,views,date_time,thumbnail_link,video_link
0,BB Ki Vines- | Ghar Kalesh |,15M views,3 months ago,https://i.ytimg.com/vi/PJ3ZQk_lr9E/hqdefault.jpg,/watch?v=PJ3ZQk_lr9E
1,BB Ki Vines- | Love Ka Hakkar |,22M views,9 months ago,https://i.ytimg.com/vi/UrIaQbIK2E4/hqdefault.jpg,/watch?v=UrIaQbIK2E4
2,BB Ki Vines Productions | Haseen Raat | Taaza ...,13M views,1 year ago,https://i.ytimg.com/vi/Nwh6CQ50h0g/hqdefault.jpg,/watch?v=Nwh6CQ50h0g
3,BB Ki Vines | Angry Masterji Part 16,29M views,1 year ago,https://i.ytimg.com/vi/HbTON0nb4DU/hqdefault.jpg,/watch?v=HbTON0nb4DU
4,BB Ki Vines Productions- Taaza Khabar | Hotsta...,20M views,1 year ago,https://i.ytimg.com/vi/zm6xa3ggt5A/hqdefault.jpg,/watch?v=zm6xa3ggt5A


In [ ]:
browser = webdriver.Chrome()
browser.get('https://www.youtube.com/')
time.sleep(3)



data = []
for link in tqdm(df['video_link']):
    link = 'https://www.youtube.com' + link
    browser.get(link)
    
    time.sleep(6) #because if the browser/driver didn't get the right amount of time to load, it will return None as value.
    
    soup = BeautifulSoup(browser.page_source, 'html.parser')
    
    try: 
        title = soup.find('yt-formatted-string', class_ = 'style-scope ytd-watch-metadata').text
    except: 
        title = np.nan
        
    try: 
        likes = soup.find('button', class_ = 'yt-spec-button-shape-next yt-spec-button-shape-next--tonal yt-spec-button-shape-next--mono yt-spec-button-shape-next--size-m yt-spec-button-shape-next--icon-leading yt-spec-button-shape-next--segmented-start').get('aria-label')
    except: 
        likes = np.nan
        
    try: 
        date_time = ' '.join(soup.find('tp-yt-paper-tooltip', class_ = 'style-scope ytd-watch-info-text').text.replace('\n', '').replace('•', '').strip().split(' ')[3:7])
    except: 
        date_time = np.nan
        
    try: 
        views = soup.find('tp-yt-paper-tooltip', class_ = 'style-scope ytd-watch-info-text').text.replace('\n', '').replace('•', '').strip().split(' ')[0]
    except: 
        views = np.nan
        
    try: 
        description = soup.find_all('span', class_ = 'yt-core-attributed-string yt-core-attributed-string--white-space-pre-wrap')[1].text.replace('\xa0', '')
    except: 
        description = np.nan
        
    data.append([title, date_time, likes, views, link, description])

In [192]:
df = pd.DataFrame(data, columns = ['title', 'date_time', 'likes', 'views', 'link', 'description'])

In [193]:
df.head()

,title,date_time,likes,views,link,description
0,BB Ki Vines- | Ghar Kalesh |,"Premiered Nov 9, 2023","like this video along with 1,717,619 other people","15,622,289",https://www.youtube.com/watch?v=PJ3ZQk_lr9E,Titu Mama comes rushing to Bhuvan's house afte...
1,BB Ki Vines- | Love Ka Hakkar |,"Premiered May 7, 2023","like this video along with 2,058,473 other people","22,138,801",https://www.youtube.com/watch?v=UrIaQbIK2E4,Sameer has fallen in love! Bhuvan and Bancho t...
2,BB Ki Vines Productions | Haseen Raat | Taaza ...,"Jan 21, 2023","like this video along with 602,157 other people","13,725,144",https://www.youtube.com/watch?v=Nwh6CQ50h0g,Watch #HotstarSpecials Taaza Khabar only on Di...
3,BB Ki Vines | Angry Masterji Part 16,"Premiered Jan 3, 2023","like this video along with 2,295,945 other people","29,934,811",https://www.youtube.com/watch?v=HbTON0nb4DU,"Bhuvan, Bablooji and Sameer visit Bubbly sir t..."
4,BB Ki Vines Productions- Taaza Khabar | Hotsta...,"Premiered Dec 13, 2022","like this video along with 661,926 other people","20,907,466",https://www.youtube.com/watch?v=zm6xa3ggt5A,Jaadu ya chamatkaar? Dhoka ya yaqeen? Vardaan ...


In [162]:
df.isnull().sum()

title          0
date_time      0
likes          0
views          0
link           0
description    0
dtype: int64

In [194]:
df.to_csv('bb_ki_vines.csv', index = False)

# Preprocessing: 

In [163]:
like = []
for i in df['likes']:
    like.append(i[27:-13])

In [164]:
df['likes'] = like

In [188]:
df['date_time'] = np.where(df['date_time'].str.startswith('Premiered '), df['date_time'].str[10:], df['date_time'])

In [ ]:
df

In [191]:
df.to_csv('bb_ki_vines_preprocessed.csv', index = False)